# BTC price prediction with Linear Regression and Support Vector Regression #
By Tanut Apiwong.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from datetime import date

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
filename = '/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-09-14.csv'
df = pd.read_csv(filename)
df.head()

## Convert timestamp to Datetime ##

In [ ]:
df.Timestamp = pd.to_datetime(df.Timestamp, unit='s')

## Set Timestamp as an index of the dataframe ##

In [ ]:
df.index = df.Timestamp
df.head()

## Resample to daily period ##

In [ ]:
df_day = df.resample('D').mean()

# Find missing values
for i, r in df_day.iterrows():
    if r.isnull().sum() > 0:
        print(i)
        print(r)
        print()

# Print the records of the date between the missing records
print()
print('Data at 2015-01-05')
print(df_day.loc['2015-01-05'])

print()
print('Data at 2015-01-09')
print(df_day.loc['2015-01-09'])

## Replace missing value with average values ##

In [ ]:
missing_replacement = df['2015-01-05': '2015-01-09'].mean(numeric_only=True)
df_day.loc['2015-01-06'] = missing_replacement
df_day.loc['2015-01-07'] = missing_replacement
df_day.loc['2015-01-08'] = missing_replacement

# Any missing value?
df_day.isnull().sum()

## Set window_size and prepare time series dataset ##

In [ ]:
window_size = (date(2020, 9, 14) - date(2020, 5, 10)).days

df_train = df_day.loc['2011-12-31':'2020-05-10']
df_test = df_day.loc['2020-05-11':]

df_train['Predicted_Price'] = df_train[['Weighted_Price']].shift(-window_size)

X_train = df_train[['Weighted_Price']].values[:-window_size].reshape(-1,1)
y_train = df_train.Predicted_Price.values[:-window_size]

y_test = df_test.Weighted_Price.values

y_test, y_test.shape

## Create SVR model and LR model ##
### Then fit with training set ###

In [ ]:
svr = SVR(kernel='rbf', C=1000)
svr.fit(X_train, y_train)

lr = LinearRegression()
lr.fit(X_train, y_train)

# Plot the SVR prediction results #

In [ ]:
plt.figure(figsize=(22, 10))
plt.plot(df_day.index.values, df_day.Weighted_Price.values, 
         label='Actual Price')
plt.plot([date(2020, 5, 10), date(2020, 5, 10)], [0, 20000], 'black', 
         label='Prediction point')

y_predict_svr = svr.predict(X_train)

plt.plot(df_train.index.values, np.append(np.zeros(window_size), y_predict_svr), 'red',
        label='Predicted price over the training set')

y_predict_svr_test = svr.predict(
    df_train[-window_size:].Weighted_Price.values.reshape(-1, 1),
)
plt.plot(df_test.index.values[:window_size], y_predict_svr_test, 'green',
        label='Predicted Price')
plt.legend()

In [ ]:
mean_squared_error(y_test, y_predict_svr_test, squared=False)

# Plot the LR prediction result #

In [ ]:
plt.figure(figsize=(22, 10))
plt.plot(df_day.index.values, df_day.Weighted_Price.values, label='Actual Price')
plt.plot([date(2020, 5, 10), date(2020, 5, 10)], [0, 20000], 'black', 
         label='Prediction point')

y_predict_lr = lr.predict(X_train)
plt.plot(df_train.index.values[window_size:], y_predict_lr, 'red', 
         label='Predicted price over the training set')

y_predict_lr_test = lr.predict(
    df_train[-window_size:].Weighted_Price.values.reshape(-1, 1)
)
plt.plot(df_test.index.values[:window_size], y_predict_lr_test, 'green', 
         label='Predicted price')
plt.legend()

## MAE of SVR and LR ##

In [ ]:
mean_squared_error(y_test, y_predict_svr_test, squared=False), mean_squared_error(y_test, y_predict_lr_test, squared=False)

## RMSE of SVR and LR ##

In [ ]:
mean_squared_error(y_test, y_predict_svr_test, squared=True), mean_squared_error(y_test, y_predict_lr_test, squared=True)

## R2 of SVR and LR ##

In [ ]:
r2_score(y_test, y_predict_svr_test), r2_score(y_test, y_predict_lr_test)